In [26]:
# import libraries
import pandas as pd
import numpy as np

In [ ]:
# specify name of csv file
filename = 'final_data.csv'

# open csv file
df = pd.read_csv(filename)

df.head()

In [ ]:
# print first 5 rows
df.rename(columns={df.columns[0]: 'index_to_drop', df.columns[1]: 'RowIndex', df.columns[2]: 'WeekIndex'},inplace=True)

# remove "index" column, since this is not the correct index
df.drop(columns=["index_to_drop"], inplace=True)

In [ ]:
# unit test that each row has a unique index
assert df['RowIndex'].nunique() == len(df.index)

# convert 'RowIndex' column to integer
df['RowIndex'] = df['RowIndex'].astype(int)

# set the 'RowIndex' column to index
df = df.set_index("RowIndex")

In [ ]:
# cut district column and insert as last column
df['District'] = df['District'].str[0:2]

# count unique categories in District column -> 11
print(df["District"].nunique())

# encode categorical variable "District" with binary encoding
# documentation: https://contrib.scikit-learn.org/category_encoders/binary.html
from category_encoders.binary import BinaryEncoder
encoder = BinaryEncoder(cols=['District'])
encoder.fit(df)
df = encoder.transform(df)

In [ ]:
# count unique categories in ZipCode "PC6" column
print(df["PC6"].nunique())

# retrieve PC4 from PC6, that is the first 4 digits of the 6 digit zip code
df["PC4"] = df["PC6"].astype(str).str[:4]
print(df["PC4"].nunique())

# encode categorical variable "PC4" with binary encoding
encoder = BinaryEncoder(cols=['PC4'])
encoder.fit(df)
df = encoder.transform(df)

In [ ]:
df.head()

In [ ]:
# all columns (including provider columns) that are not in the above lists are unknown variables
providers = ["Total", "Vattenfall", "Allego", "Equans", "EvBox", "Nuon", "WDS", "Pitpoint", "Ecotap", "Engie"]

# create a new column called "number of chargers" which is the sum of the values of the columns which names are stored in the providers list
df['number_of_chargers'] = df[providers].sum(axis=1)

providers.append("number_of_chargers")

# reverse list
providers = providers[::-1]

In [ ]:
# split columns into several groups: identifiers, target variables, know_variables, unknown_variables, purely_descriptive_variables

# reorder columns in the following order:
identifiers = ["PC6", "Date"]
know_variables = ["WeekIndex", "MaxPower"]
target_variables = ["kWh", "Blocked_kWh"]

purely_descriptive_variables = ["ChargeSocket_ID_count", "ConnectionTimeHours", "power", "effective_charging_hrs", "MaxOccupancy", "SpareCap_Effective", "SpareCap_Occup_kWh", "SpareCap_Hrs", "Effective%", "Occupancy_kwh%"]


unknown_variables = [col for col in df.columns if col not in identifiers + target_variables + know_variables + providers+ purely_descriptive_variables]
unknown_variables = providers + unknown_variables

print(len(identifiers + target_variables + know_variables + purely_descriptive_variables + unknown_variables))

In [ ]:
# sort df columns in the following order: identifiers, target variables, know_variables, unknown_variables, purely_descriptive_variables
df = df[identifiers + know_variables + target_variables + unknown_variables + purely_descriptive_variables]

In [ ]:
# inspect df without the purely_descriptive_variables
df[identifiers + know_variables + target_variables + unknown_variables].head()

In [ ]:
# check missing values: show only columns where missing values >0 and their count per column
print(df.isna().sum()[df.isna().sum() > 0])

In [ ]:
# sort df by date column
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by=['PC6', 'Date'])

In [ ]:
# add week of year column, extracted from date column
df["WeekOfYear"] = pd.to_datetime(df['Date']).dt.isocalendar().week

# month of year
df["MonthOfYear"] = pd.to_datetime(df['Date']).dt.month

# sine and cosine week of year -> to introduce cyclical nature of months
# sine & cosine week of year
# cosine week of year
df["CosWeekOfYear"] = np.cos(2*np.pi*df["WeekOfYear"]/52)

# sine week of year
df["SinWeekOfYear"] = np.sin(2*np.pi*df["WeekOfYear"]/52)

# cosine month of year
df["CosMonthOfYear"] = np.cos(2*np.pi*df["MonthOfYear"]/12)

# sine month of year
df["SinMonthOfYear"] = np.sin(2*np.pi*df["MonthOfYear"]/12)

In [ ]:
df.head()

In [ ]:
# count number of remaining rows
print(df.shape)

In [ ]:
# load cleaned data
# df = pd.read_csv(filename.split('.')[0] + '_cleaned.csv')

# create new column as concatenation of PC6 and WeekIndex columns
df['PC6_WeekIndex'] = df['PC6'].astype(str) + df['WeekIndex'].astype(str)

# move this column to the first position
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]

# count duplicates in PC6_WeekIndex column
df['PC6_WeekIndex'].duplicated().sum()

In [ ]:
# keep only first 10000 rows for testing purposes
# df = df.head(10000)

# inspect data
df.head()

In [ ]:
# save to new csv file named original name + _cleaned
df.to_csv(filename.split('.')[0] + '_cleaned.csv', index=False)